In [1]:
# I feel like this entire file is sort of broken, maybe it's better to use MM_Covid_Explain2??
import re
import simdjson as sj
import gzip
from tqdm import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import swifter
import seaborn as sns
import numpy as np	
from nltk import sent_tokenize
import re
from sentence_transformers import SentenceTransformer
from wutils.general import save_pickle, load_pickle
from wutils.mat import MarkedMatrix
from scipy.spatial.distance import cdist
from sklearn.linear_model import LogisticRegression

In [2]:
# changed from ./pickles/who_cord_df.pkl to ./pickles/newest_cord_df_ft.pkl
cord_df = load_pickle('./pickles/newest_cord_df_ft.pkl')

In [3]:
# changed from ../who to ./pickles/mm_df.pkl
mm_df = load_pickle('./pickles/mm_df.pkl')

In [416]:
# cord_df[cord_df.abstract.swifter.apply(lambda x: 'shortages' in x)]

In [5]:
# changed from ./pickles/newest_good_sample_df_ft.pkl to ./pickles/clean_good_df.pkl
good_df = load_pickle('./pickles/clean_good_df.pkl')

In [6]:
# changed from ./pickles/newest_bad_df_ft.pkl to ./pickles/clean_bad_df.pkl
bad_df = load_pickle('./pickles/clean_bad_df.pkl')

In [7]:
import urllib.parse
def get_hostname(url):
    o = urllib.parse.urlsplit(url)
    return o.hostname

In [8]:
good_df['hostname'] = good_df['url'].swifter.apply(get_hostname)
bad_df['hostname'] = bad_df['url'].swifter.apply(get_hostname)

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

In [9]:
good_df.hostname.value_counts()

www.reuters.com           5612
www.thehindu.com          5139
www.forbes.com            3367
www.france24.com          3364
indianexpress.com         3121
                          ... 
tucson.com                   1
draftwire.usatoday.com       1
www.spokesman.com            1
uk.reuters.com               1
gmat.economist.com           1
Name: hostname, Length: 238, dtype: int64

In [10]:
good_df.hostname.value_counts()

www.reuters.com           5612
www.thehindu.com          5139
www.forbes.com            3367
www.france24.com          3364
indianexpress.com         3121
                          ... 
tucson.com                   1
draftwire.usatoday.com       1
www.spokesman.com            1
uk.reuters.com               1
gmat.economist.com           1
Name: hostname, Length: 238, dtype: int64

In [9]:
# target_df = bad_df[bad_df.hostname == 'beforeitsnews.com']
# target_df = target_df[target_df.covidFreq > 20]

In [11]:
target_df = good_df[good_df.hostname == 'www.reuters.com']

In [12]:
target_df.sentences.iloc[505]

['Military personnel are seen testing people at a coronavirus test centre in the car park of Chessington World of Adventures as the spread of the coronavirus disease (COVID-19) continues, London, Britain, April 14, 2020.',
 '/Peter Nicholls\nLONDON () - Britain is working towards including coronavirus deaths which take place outside of hospital in its daily figures, Public Health England Medical Director Yvonne Doyle said on Tuesday.',
 '“We just need to be absolutely clear that the cause of death that is attributed is correct and that is what takes time on the death certificate to get right,” she said.',
 '“But we would like to have much more rapid data, preferably on a daily basis, and that is what we are working towards.”']

In [13]:
cord_df

,cord_uid,sha,doi,title,abstract,journal,TOT_CITES,IMPACT_FACTOR,ARTL_INFLUENCE,CATEGORY_DESCRIPTION,JIF_PERCENTILE,title_embeddings,abstract_embeddings,title_and_abstract_embeddings,sentences,sent_embeddings,ft_embeddings
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,10.1186/1471-2458-3-5,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",BMC PUBLIC HEALTH,36306.0,2.567,1.042,"PUBLIC, ENVIRONMENTAL & OCCUPATIONAL HEALTH",68.548387,"[0.22491726279258728, -0.015980135649442673, -...","[0.1403646022081375, -0.4500912129878998, -0.1...","[0.16280563175678253, -0.4352791905403137, -0....",[Airborne rhinovirus detection and effect of u...,"[[-0.5930148, -0.61757505, 0.26939434, -0.2021...","[[0.025865793, 0.0144151505, 0.024386879, -0.0..."
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,10.1186/gb-2003-4-5-213,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...,GENOME BIOL,NaN,NaN,NaN,NaN,NaN,"[0.06675072759389877, -0.044052284210920334, -...","[-0.25839436054229736, -0.10324420779943466, -...","[-0.19375188648700714, -0.1265585720539093, -0...",[Discovering human history from stomach bacter...,"[[-0.6388278, -0.49111003, -0.2363218, -0.8208...","[[0.0047203675, 0.041005637, -0.013781068, 0.0..."
2,le0ogx1s,NaN,10.1186/gb-2003-4-7-113,A new recruit for the army of the men of death,"The army of the men of death, in John Bunyan's...",GENOME BIOL,NaN,NaN,NaN,NaN,NaN,"[-0.5324184894561768, -0.0358302928507328, -0....","[-0.281924307346344, -0.21824604272842407, -0....","[-0.49570220708847046, -0.2127537578344345, -0...",[A new recruit for the army of the men of deat...,"[[0.0010873515, -0.6144223, 0.026371827, -1.18...","[[0.028263845, -0.10034516, -0.036279414, -0.0..."
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,10.1186/1471-2350-4-9,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,BMC MED GENET,NaN,NaN,NaN,NaN,NaN,"[0.25805529952049255, 0.059809811413288116, -0...","[0.003256639465689659, -0.18425962328910828, -...","[0.00840730220079422, -0.17367023229599, -0.30...",[Association of HLA class I with severe acute ...,"[[-0.19145542, -0.9727794, 0.14144804, -1.1873...","[[0.033887863, -0.05646804, 0.0014032094, 0.00..."
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,10.1186/1471-2334-3-19,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,BMC INFECT DIS,NaN,NaN,NaN,NaN,NaN,"[-0.2515140473842621, -0.04684412479400635, -0...","[0.022290252149105072, -0.3348672688007355, -0...","[-0.01470943819731474, -0.32205796241760254, -...",[A double epidemic model for the SARS propagat...,"[[-0.4014988, -1.481686, 0.44280347, -0.390691...","[[0.018823104, -0.095268644, -0.0057887835, -7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50463,5zmgvlj9,f3b4724163d8a42bc2788547d18f2521e14c44a1,10.1016/j.idc.2016.07.004,Health Care–Acquired Viral Respiratory Diseases,Health care–acquired viral respiratory infecti...,INFECTIOUS DISEASE CLINICS OF NORTH AMERICA,2765.0,4.757,1.657,INFECTIOUS DISEASES,84.831461,"[-0.04793879762291908, -0.23308999836444855, -...","[0.13221529126167297, -0.06348022818565369, -0...","[0.12945394217967987, -0.10145720839500427, -0...",[Health Care–Acquired Viral Respiratory Diseas...,"[[0.36586708, -0.37732273, 0.04530729, -0.8243...","[[0.085807346, 0.052655112, 0.052195895, 0.003..."
50464,6lh4jsor,93244303c6041ad53fcc59cc06a03cc51621edcc,10.1016/j.antiviral.2007.10.006,NIAID resources for developing new therapies f...,"Abstract Severe viral infections, including he...",ANTIVIRAL RESEARCH,8440.0,4.130,1.137,PHARMACOLOGY & PHARMACY,81.460674,"[0.006424863822758198, -0.2415492832660675, -0...","[-0.12714576721191406, -0.10232926905155182, -...","[-0.09399458020925522, -0.10015833377838135, -...",[NIAID resources for developing new therapies ...,"[[-0.37147185, -0.6980188, 0.43672624, -

In [15]:
# missing this ./data/words.txt file
# It's probably ok though, I think this is just used to show some data
with open('./data/words.txt') as f:
    words = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: './data/words.txt'

In [17]:
# needs words.txt file
words = [x.strip().lower() for x in words]

NameError: name 'words' is not defined

In [19]:
# needs words.txt file
import random
sents = []
for i in range(5000):
    n_words = random.randint(10, 30)
    sent_words = random.choices(words, k=n_words)
    sent = ' '.join(sent_words)
    sent = sent[0].upper() + sent[1:] + '.'
    sents.append(sent)

NameError: name 'words' is not defined

In [20]:
# this works, but for the next function you need sents which needs words.txt
sent_encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [21]:
encoded = sent_encoder.encode(sents, convert_to_tensor=True, show_progress_bar=True)

Batches: 0it [00:00, ?it/s]

RuntimeError: stack expects a non-empty TensorList

In [22]:
# still need words.txt here
encoded = encoded.numpy()

NameError: name 'encoded' is not defined

In [394]:
# true_df['text'].swifter.apply(lambda x: 'myth' in x.lower()).sum()

In [23]:
# this works ok tho
KW = 'vaccine'
print(good_df['cleanText'].swifter.apply(lambda x: KW in x.lower()).sum())
print(bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower()).sum())

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

7647


Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

2735


In [323]:
# vax_good_df = good_df.sample(n=20000).dropna()

In [324]:
# vax_bad_df = bad_df.sample(n=20000).dropna()

In [24]:
# works ok as well
vax_good_df = good_df[good_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
vax_bad_df = bad_df[bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]

Pandas Apply:   0%|          | 0/97539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

In [25]:
vax_good_df['mean_embed'] = vax_good_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
vax_bad_df['mean_embed'] = vax_bad_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
# vax_good_df['mean_embed'] = vax_good_df['ft_embeddings'].swifter.apply(lambda x: np.array([y for y in x if np.isfinite(y).all()]).mean(axis=0))
# vax_bad_df['mean_embed'] = vax_bad_df['ft_embeddings'].swifter.apply(lambda x: np.array([y for y in x if np.isfinite(y).all()]).mean(axis=0))

Pandas Apply:   0%|          | 0/7647 [00:00<?, ?it/s]

/var/folders/ms/0t06g66d4cncxmtwvsxtqjh40000gn/T/ipykernel_16501/3475759766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vax_good_df['mean_embed'] = vax_good_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))


Pandas Apply:   0%|          | 0/2735 [00:00<?, ?it/s]

/var/folders/ms/0t06g66d4cncxmtwvsxtqjh40000gn/T/ipykernel_16501/3475759766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vax_bad_df['mean_embed'] = vax_bad_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))


In [26]:
vax_good_mat = np.vstack(vax_good_df.mean_embed)
vax_bad_mat = np.vstack(vax_bad_df.mean_embed)

In [27]:
vax_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

In [28]:
np.where(~np.isfinite(vax_mm.mat))

(array([], dtype=int64), array([], dtype=int64))

In [30]:
# this was originally after the previous method
# but should really be before, since we are initializing model
model = LogisticRegression(max_iter=400)

In [31]:
vax_mm.single_split_classify(model)

Training model...
Evaluating model...


/Users/toasterdoodle/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


((0.8416795069337443, 0.8986436498150431), {'bad': 0, 'good': 1})

In [32]:
cord_df = cord_df.dropna()

In [33]:
has_vax = cord_df[cord_df.title.swifter.apply(lambda x: KW in x.lower()) | cord_df.abstract.swifter.apply(lambda x: KW in x.lower())]
# has_vax = cord_df.sample(n=5000)

Pandas Apply:   0%|          | 0/10149 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10149 [00:00<?, ?it/s]

In [34]:
has_vax['mean_embed'] = has_vax['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
# has_vax['mean_embed'] = has_vax['ft_embeddings'].swifter.apply(lambda x: np.array(x).mean(axis=0))

Pandas Apply:   0%|          | 0/1315 [00:00<?, ?it/s]

/var/folders/ms/0t06g66d4cncxmtwvsxtqjh40000gn/T/ipykernel_16501/1907037509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_vax['mean_embed'] = has_vax['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))


In [35]:
vax_ref = np.vstack(has_vax['mean_embed'])

In [36]:
vax_ref.shape

(1315, 768)

In [37]:
vax_good_mat.shape

(7647, 768)

In [38]:
# vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')

In [39]:
# vgood2ref = cdist(vax_good_mat, encoded, 'cosine')
# print('done w/ good')
# vbad2ref = cdist(vax_bad_mat, encoded, 'cosine')

In [40]:
# can find replacements for 'cosine' here
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')
print('done w/ good')
vbad2ref = cdist(vax_bad_mat, vax_ref, 'cosine')

done w/ good


In [41]:
vax_ref[~np.isfinite(vax_ref)] = 0

In [42]:
vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

In [43]:
vax_raw_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

In [44]:
# loading more pickles here
# actually don't need to replace this old pickle here
# this is the pickle for labelled claims with which to test how well our algorithm is performing
# I am assuming
claims_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')
claims_df = claims_df.dropna()

# sorting out the claims
true_claims_df = claims_df[claims_df['label'] == 'true']
false_claims_df = claims_df[claims_df['label'] == 'false']
misleading_claims_df = claims_df[claims_df['label'] == 'misleading']

true_claims_mat = np.vstack(list(true_claims_df['bert_embedding']))
false_claims_mat = np.vstack(list(false_claims_df['bert_embedding']))

In [46]:
# ok so here we are finding the difference between the the matrix of true claims and the reference
# using cosine
true_claims2ref = cdist(true_claims_mat, vax_ref, 'cosine')
# and here we are finding the difference between the matrix of false claims and the reference
# using cosine
false_claims2ref = cdist(false_claims_mat, vax_ref, 'cosine')

In [47]:
# ok what the fuck?
# I thought we just calculated this
# I'm confused as to why this cell exists
# also why is it negative now
true_claims2ref = -cdist(true_claims_mat, vax_ref)
false_claims2ref = -cdist(false_claims_mat, vax_ref)

In [48]:
# previously: (51, 4686)
# mine: (51, 1315)
# I think this is fine since the 51 is still the same
# I think 1315 vs 4686 simply means I probably got a smaller pickle or something
true_claims2ref.shape

(51, 1315)

In [50]:
# ohhh, are we calculating how many we are classifying correctly now?
# like how many in the true_claims_mat are actually true
# and vice versa for false
# also I looked up what this marked matrix thing is out of curiosity
# interestingly, it's a python library that Will himself wrote
# I guess it's just a matrix with a label?
claims_mm = MarkedMatrix([('true', true_claims_mat), ('false', false_claims_mat)])
claims_mm2covid = MarkedMatrix([('true', true_claims2ref), ('false', false_claims2ref)])

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
# tmodel = DecisionTreeClassifier()
# oh bruh what the, why is tmodel defined over here?
tmodel = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [55]:
# ayo what the where is this tmodel defined??
# ok I found it, I'm going to move it to before this
claims_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.90625, 0.9508196721311475), {'true': 0, 'false': 1})

In [56]:
tmodel_claims = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [57]:
claims_mm2covid.single_split_classify(tmodel_claims, return_labels=True)

Training model...
Evaluating model...


((0.921875, 0.959349593495935), {'true': 0, 'false': 1})

In [60]:
from treeinterpreter import treeinterpreter as ti

In [65]:
# yo where is ti?
# we need ti.predict here
# it is nowhere, what is this ti and how do I import this
# ok so after some searching, it seems ti is treeinterpreter
# I'm going to import this here but in the cell before
def tree_claim_explain_bad(idx):
    # guess this shows why this document should be considered "false"
    pred, bias, contrib = ti.predict(tmodel_claims, false_claims2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', false_claims_df.iloc[idx].text)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_good = np.argsort(cmat[:, 0])[::-1]
    import_bad = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

def tree_claim_explain_good(idx):
    # tries to explain why this document shold be considered "true"
    pred, bias, contrib = ti.predict(tmodel_claims, true_claims2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', true_claims_df.iloc[idx].text)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_good = np.argsort(cmat[:, 0])[::-1]
    import_bad = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [66]:
# works now, nice
tree_claim_explain_good(10)

CLEAN TEXT:  Schools have resumed in New Brunswick despite the COVID-19 pandemic
Prediction:  [[0.722 0.278]]
Bias:  [[0.10730026 0.89269974]]
===== Towards Good: =====
---------------------------------------------
CONTRIB: [ 0.0453166 -0.0453166]
TITLE: Towards antivirals against chikungunya virus
---------------------------------------------
CONTRIB: [ 0.03638032 -0.03638032]
TITLE: The concept of quarantine in history: from plague to SARS
---------------------------------------------
CONTRIB: [ 0.02186464 -0.02186464]
TITLE: Pandemic H1N1 in Canada and the use of evidence in developing public health policies – A policy analysis
---------------------------------------------
CONTRIB: [ 0.02170376 -0.02170376]
TITLE: Dairy calf management, morbidity and mortality in Ontario Holstein herds. III. Association of management with morbidity
---------------------------------------------
CONTRIB: [ 0.01935542 -0.01935542]
TITLE: How Should the Rehabilitation Community Prepare for 2019-nCoV?
==

In [64]:
tree_claim_explain_bad(11)

CLEAN TEXT:  COVID-19 testing patent was submitted in 2015 and COVID19 is a scam
Prediction:  [[0.046 0.954]]
Bias:  [[0.10730026 0.89269974]]
===== Towards Bad: =====
---------------------------------------------
CONTRIB: [-0.00172305  0.00172305]
ABSTRACT: Abstract Background Hajj is the world’s largest annual mass gathering that attracts two to three million Muslims from around the globe to a religious assemblage in Makkah, Saudi Arabia. The risk of acquisition and transmission of influenza among Hajj pilgrims is high. Therefore, influenza vaccination is recommended, and was monitored frequently among pilgrims from different countries. However, the vaccination uptake among Saudi pilgrims has not been assessed in recent years. Objective This analysis aims to evaluate influenza vaccine uptake among Saudi Hajj pilgrims, and identify the key barriers to vaccination. Method Data on influenza vaccination were obtained from Saudi pilgrims who took part in a large trial during the Hajj of 2

In [67]:
vax_mm.mat[~np.isfinite(vax_mm.mat)] = 0

In [68]:
vax_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.7916024653312789, 0.8715879420840256), {'bad': 0, 'good': 1})

In [69]:
vax_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.8039291217257319, 0.879867831012509), {'bad': 0, 'good': 1})

In [ ]:
tmodel2 = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [ ]:
vax_raw_mm.single_split_classify(tmodel2, return_labels=True)

Training model...
Evaluating model...


((0.8081664098613252, 0.8815413891531874), {'bad': 0, 'good': 1})

In [72]:
# ok so we had tree_claim explain before
# and now we need a separate function for tree explain?
# they look almost identical though
# but obviously I'm not going to question Will's handiwork without knowing anything LOL
def tree_explain_good(idx):
    pred, bias, contrib = ti.predict(tmodel, vgood2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_good_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

def tree_explain_bad(idx):
    pred, bias, contrib = ti.predict(tmodel, vbad2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_bad_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [73]:
tree_explain_bad(1917)
# this clean text is WILD LMAO
# "California is a bastion of dark blue communist totalitarian madness"
# this is actually hilarious
# GOOD WORK WILL, I SALUTE YOU
# GOOD WORK DETECTING THIS AS NONSENSE
# this does make me wonder though, do certain individual words have a higher chance of indicating a statement as false?
# like the word "communist" or "totalitarian" for example

CLEAN TEXT:  California is a bastion of dark blue communist totalitarian madness, and the governor, Gavin Newsom (or as I like to say, Gavin Gruesome) has proclaimed himself dictator. He likes to say things like “we have to enforce, and we will,” and “whether you like it or not,” and he even believes he has the authority to dictate to you what you can do at your backyard bar-b-ques on Independence Day.
Last night I grabbed a bite to eat at a local restaurant because due to Newsom’s recent executive order, inside dining is illegal, again, or at least it will be as of tomorrow. Why? Because according to Newsom, we have a spike in cases of COVID-19 in this State, and it’s because too many people are leaving their homes, and because too many people are not wearing masks in public. And this time, he assured us, he won’t be relaxed and let renegade counties like Riverside County get away with not enforcing his edict. “We have to enforce, and we will.”
Newsom has repeatedly told us that “scie

In [75]:
vax_good_df.iloc[157].cleanText

'A Halifax-based research company has developed a COVID-19 antibody test.\nMedMira hopes to be able to market it to countries ready to begin surveillance programs to find out how widespread infections are within their borders.\nThe company developed one of the first rapid detection kits for HIV.\nOther Canadian companies are already making similar tests and selling them abroad.\nHermes Chan, the company\'s co-founder, said his company is counting on two things to help it beat its competitors — speed and ease of operation.\n"From taking a drop of blood specimen to complete one single test, it will take less than three minutes," said Chan.\nSimilar to pregnancy test\nAccording to Chan, other tests can take 15-20 minutes to complete and some need a reader to interpret the results.\nMedMira\'s process, which is similar to a pregnancy test, registers a positive or negative result on the testing device itself.\nChan said lab technicians could also perform multiple tests simultaneously, rathe

In [77]:
tree_explain_good(157)

CLEAN TEXT:  A Halifax-based research company has developed a COVID-19 antibody test.
MedMira hopes to be able to market it to countries ready to begin surveillance programs to find out how widespread infections are within their borders.
The company developed one of the first rapid detection kits for HIV.
Other Canadian companies are already making similar tests and selling them abroad.
Hermes Chan, the company's co-founder, said his company is counting on two things to help it beat its competitors — speed and ease of operation.
"From taking a drop of blood specimen to complete one single test, it will take less than three minutes," said Chan.
Similar to pregnancy test
According to Chan, other tests can take 15-20 minutes to complete and some need a reader to interpret the results.
MedMira's process, which is similar to a pregnancy test, registers a positive or negative result on the testing device itself.
Chan said lab technicians could also perform multiple tests simultaneously, rath

In [78]:
tree_explain_good(103)

CLEAN TEXT:  Only two states in the U.S. have reported a decline in coronavirus cases in the last week.
Across the country, case numbers have risen along with hospitalizations due to COVID-19, according to an article published by .
Wisconsin health officials reported a record high of 141 new hospitalizations on Wednesday, according to data obtained by the outlet.
MADISON, Wis. (AP) — Wisconsin governor activates 530-bed field hospital as coronavirus spread strains capacity in state facilities. — Zeke Miller (@ZekeJMiller) October 7, 2020
“We obviously hoped this day wouldn’t come, but unfortunately, Wisconsin is in a much different and more dire place today, and our healthcare systems are being overwhelmed,” Gov. Tony Evers said while announcing the opening of a field hospital, according to . (RELATED: New Coronavirus Restrictions Enacted Across Europe As Coronavirus Cases Rise)
Utah Lt. Gov. Spencer Cox claimed the spread of the virus in the state is “way too high” and confirmed the h

In [79]:
# bro there's no way he imports treeinterpreter all the way down here
# I'm commenting this out
# but I won't remove it, just to show how wild of a place it was put in
# from treeinterpreter import treeinterpreter as ti

In [80]:
pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :].reshape((1, -1)))
# pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :])

In [81]:
cmat = contrib.reshape((-1, 2))
import_bad = np.argsort(cmat[:, 0])[::-1]
import_good = np.argsort(cmat[:, 1])[::-1]

In [82]:
print(import_bad[:5])

[1299  902  575  935  477]


In [84]:
# original: [1141  821 2810 2357 1929]
# mine: [1166  744 1073  245 1132]
print(import_good[:5])

[1166  744 1073  245 1132]


In [86]:
# this doesn't work
# I don't know why
# and I'm not going to try to figure out why
has_vax.iloc[2810].abstract

IndexError: single positional indexer is out-of-bounds

In [87]:
# ok so this cell was never run before
# when I got it
# guess I'll run it then
for k in range(contrib.shape[0]):
    for c, feature in zip(contrib[k], [f'doc{x}' for x in range(4686)]):
        if abs(c[0] - c[1]) < 0.000001:
            continue
        print(feature, c, c[0] - c[1])

doc0 [-0.00059915  0.00059915] -0.0011983001306000384
doc1 [-0.00020173  0.00020173] -0.00040345313601127485
doc2 [-0.00026348  0.00026348] -0.0005269665150618747
doc3 [-0.00011691  0.00011691] -0.00023382849773185738
doc4 [ 0.00013405 -0.00013405] 0.00026809269162210333
doc6 [-0.00050236  0.00050236] -0.0010047282552870115
doc7 [-0.00030725  0.00030725] -0.0006144903464860804
doc8 [-0.00029211  0.00029211] -0.0005842254363993497
doc9 [ 0.00032696 -0.00032696] 0.0006539198278522752
doc10 [-0.00017239  0.00017239] -0.00034478974076105343
doc11 [ 7.7358525e-05 -7.7358525e-05] 0.0001547170499357669
doc12 [-0.00078144  0.00078144] -0.001562883469261043
doc13 [ 0.0006663 -0.0006663] 0.0013325994643324371
doc14 [ 0.00258492 -0.00258492] 0.005169834979817087
doc15 [-0.00051341  0.00051341] -0.0010268248443567637
doc16 [-0.00055449  0.00055449] -0.0011089894833653533
doc17 [-0.00032726  0.00032726] -0.0006545107435590937
doc18 [-0.00071608  0.00071608] -0.0014321522561454155
doc19 [-0.00076866

In [88]:
contrib[0].shape
# ok, from this point on, pretty much none of the cells are run
# maybe Will said fuck it and decided to make a new file instead LOl

(1315, 2)

In [89]:
vax_mm.mat.shape

(10382, 1315)

In [90]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [91]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):\n  path = []".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            print("{}  print('{} <= {}')".format(indent, name, threshold))
            print("{}  path.append(({}, '<=', {}))".format(indent, name[3:], threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            print("{}  print('{} > {}')".format(indent, name, threshold))
            print("{}  path.append(({}, '>', {}))".format(indent, name[3:], threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
#             print(tree_.value[node].shape)
            print("{}return ({}, path)".format(indent, list(tree_.value[node].ravel())))

    recurse(0, 1)


In [ ]:
def explain_tree(idx):
    (res, path) = tree(*vbad2ref[idx, :])
    print('RESULT: {}, {}'.format(res, tmodel.predict(vbad2ref[idx, :].reshape((1, -1)))))
    print('Clean Text: {}'.format(vax_bad_df.iloc[idx].cleanText))
    for term in path:
        name, op, thresh = term
        print('-'*25)
        print('CRITERIA: ({}): {} {}'.format(name, op, thresh))
        print('TITLE: {}'.format(has_vax.iloc[name].title))
        print('ABSTRACT: {}'.format(has_vax.iloc[name].abstract))

In [ ]:
explain_tree(310)

In [ ]:
tree_to_code(tmodel, [f'doc{x}' for x in range(4686)])

In [ ]:
tree(*vgood2ref[254, :])

In [ ]:
tree(*vgood2ref[255, :])

In [ ]:
vax_good_df.iloc[255].cleanText

In [ ]:
has_vax.iloc[4460].abstract

In [ ]:
plot_tree(tmodel)

In [ ]:
s_cord_df = cord_df.sample(n=3000)
good_mat = 

In [ ]:
def get_results(num, k=5):
    res = model.predict(vgood2ref[num, :].reshape(1, -1))
    print(res)
    weights = model.coef_ * vgood2ref[num, :].T
#     top_k = np.argsort(weights)[0, :k]
    top_k = np.argsort(weights)[0, :k]
    print('Clean Text: ', vax_good_df.iloc[num].cleanText)
    for el in top_k:
        print('-'*25)
        print(f'Weight: {weights[0, el]}')
        print(f'Related abstract ({el}): {has_vax.iloc[el].abstract}')
        print(f'Related title ({el}): {has_vax.iloc[el].title}')

In [ ]:
# get_results(2912)

In [ ]:
get_results(129)

In [ ]:
[(model.coef_[0, x], has_vax.iloc[x].abstract) for x in np.argsort(model.coef_[0, :])[::-1][:10]]

In [ ]:
cord_df

In [ ]:
vax_bad_df.iloc[0].mean_embed.shape

In [ ]:
false_df['text'].swifter.apply(lambda x: 'myth' in x.lower()).sum()

In [ ]:
false_df = claims_df[claims_df['label'] == 'false']
true_df = claims_df[claims_df['label'] == 'true']

In [ ]:
s_mat.iloc[0].shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import cdist

In [ ]:
true_mat = np.vstack(true_df.bert_embedding)
fake_mat = np.vstack(false_df.bert_embedding)

In [ ]:
best_model = None
best_f1 = 0

for i in tqdm(range(1000)):
    s_cord = cord_df.sample(100)
    s_mat = np.vstack(s_cord['sent_embeddings'].swifter.progress_bar(False).apply(lambda x: x.mean(axis=0)))
    model = LogisticRegression(max_iter=1000)
    true2s = cdist(true_mat, s_mat, 'cosine') # try cosine?
    fake2s = cdist(fake_mat, s_mat, 'cosine')
    mm = MarkedMatrix([('true', true2s), ('fake', fake2s)])
    acc, f1 = mm.single_split_classify(model, verbose=False)
    if f1 > best_f1:
        best_f1 = f1
        best_model = model
        print('New best F1: ', best_f1)

In [ ]:
best_f1

In [ ]:
cnt = 0
for x in np.argsort(model.coef_[0, :])[::-1]:
    print(s_cord.iloc[x].title)
    cnt += 1
    if cnt > 1000:
        break

In [ ]:
claim_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')

In [ ]:
claim_df

In [ ]:
claims_mat = np.array(list(claim_df['bert_embedding']))

In [ ]:
target_df.iloc[11].cleanText

In [ ]:
bad_df[bad_df.hostname == 'beforeitsnews.com'].iloc[4].cleanText

In [ ]:
mm_df = load_pickle('./pickles/mm_df.pkl').dropna()

In [ ]:
cord_df

In [ ]:
cord_df['mean_embed'] = cord_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

In [ ]:
s_cord = cord_df.sample(n=100)

In [ ]:
has_vax = cord_df['abstract'].swifter.apply(lambda x: 'vaccine' in x)

In [ ]:
s_cord = cord_df[has_vax]

In [ ]:
mat_cord = np.vstack(s_cord['mean_embed'])

In [ ]:
real_df = mm_df[mm_df['label'] == 'real']
fake_df = mm_df[mm_df['label'] == 'fake']

In [ ]:
fake_df = fake_df[fake_df.text != 'NONE']

In [ ]:
good_mat = np.vstack(real_df['mean_embed'])
bad_mat = np.vstack(mm_df[mm_df['label'] == 'fake']['mean_embed'])

In [ ]:
mm = MarkedMatrix((('real', good_mat), ('fake', bad_mat)))

In [ ]:
good2cord = cdist(good_mat, mat_cord, 'cosine')
bad2cord = cdist(bad_mat, mat_cord, 'cosine')

In [ ]:
mm = MarkedMatrix((('real', good2cord), ('fake', bad2cord)))

In [ ]:
model = LogisticRegression(max_iter=1000, verbose=1, n_jobs=16, solver='lbfgs')

In [ ]:
mm.single_split_classify(model)

In [ ]:
def get_results(num, k=5):
    res = model.predict(bad2cord[num, :].reshape(1, -1))
    print(res)
    weights = model.coef_ * bad2cord[num, :].T
#     top_k = np.argsort(weights)[0, :k]
    top_k = np.argsort(weights)[0, :k]
    print('Clean Text: ', fake_df.iloc[num].text)
    for el in top_k:
        print('-'*25)
        print(f'Weight: {weights[0, el]}')
        print(f'Related abstract ({el}): {cord_df.iloc[el].abstract}')
        print(f'Related title ({el}): {cord_df.iloc[el].title}')

In [ ]:
get_results(930, k=10)

# 